In [22]:
"""This class converts a dict to nested objects"""

class Struct(object):
    """
    Attributes will depend on the structure of object. 
    If we keep calling the 'everything' newsapi, then the attributes will be:
    
                    articles: A list of articles, each with their own objects
                    status: Status of request, should be 'ok'
                    totalResults: The total number of results available for the request, will need
                                  to use the &page= parameter to get these as only 20 articles are
                                  returned per request.
                                  
    Resource: https://stackoverflow.com/questions/1305532/convert-python-dict-to-object
    """
    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)): 
            return type(value)([self._wrap(v) for v in value])
        else:
            return Struct(value) if isinstance(value, dict) else value


"""
General class to aggregate all useful objects. 
Could customize, e.g. change structure of get_raw_data to affect data object
"""

class myclass(object):
    """
    Attributes:
                call: The url sent to newsapi
                raw_data: The dictionary returned when requesting call
    """
    
    def get_raw_data(self,call):
        r = requests.get(call).json()
        
        for i in r['articles']:
            del i['author']
            del i['urlToImage']
            t = datetime.datetime.strptime(i['publishedAt'], "%Y-%m-%dT%H:%M:%S%fZ")
            nt = t.replace(hour=0, minute=0, second=0, microsecond=0)
            i['publishedAt'] = str(nt)
            i['source'] = i['source']['name']
        
        return r
    
    def __init__(self,call):
        self.call = call
        self.data = Struct(self.get_raw_data(call))
        self.data.n_pages = self.data.totalResults/20
        
    """
    Takes call and paginates over user input number of pages to provide a list of 
    lists made up of articles
    """
        
    def paginate(self,n):
        #If page argument already exists in call, remove it
        fp = self.call.find('&page=')
        if fp > 0:
            l = [x for x, v in enumerate(self.call) if v == '&']
            l.append(len(self.call))
            nxt = l[next(x[0] for x in enumerate(l) if x[1] > fp)]
            base_call = self.call[:fp] + self.call[nxt:]
        else:
            base_call = self.call
            
        #print(base_call)
        articles_list = []
        
        for i in range(1,n+1):
            new_call = base_call + "&page=" + str(i)
            #print(new_call)
            d = Struct(self.get_raw_data(new_call))
            #print(d.articles[0].__dict__)
            articles_list.extend(d.articles)
            
        return articles_list


In [23]:
import pickle
PIK = "test.dat"

with open(PIK, "rb") as f:
    data = pickle.load(f)

In [24]:
len(data)

5000

In [25]:
def manually_classify(l):
    print('Enter the polarity for the diplayed title and description, p for postive, p, for negative, nn for neutral.')
    print('Enter "stop" if you wish to stop. \n')
    for i in range(0,len(l)):
        print("\n")
        print(l[i].title)
        print(l[i].description)
        print("\n")
        while True:
            p = input('What is the polarity of this article? ')
            if p not in ('p', 'n', 'nn', 'stop'):
                print("\n")
                print("Not a valid response, please try again.")
            else:
                break
                
        if p == 'stop':
            break
        else:
            l[i].polarity = p


In [18]:
manually_classify(data)

Enter the polarity for the diplayed title and description, p for postive, p, for negative, nn for neutral.
Enter "stop" if you wish to stop. 



Why Bitcoin’s Price Is So Volatile
In 2017, Bitcoin ’s value soared from $1,000 to just under $20,000 before dropping down to around $13,000 at the end of the year. Since then, it’s value has risen and dropped sporadically from day to day, dragging smaller cryptocurrencies like Ether and Rippl…


What is the polarity of this article? n


How to Buy Cryptocurrency
So you’re ready to buy some cryptocurrency. Maybe you’ve been reading up on blockchain technology and you’re convinced it really is the future. Or maybe you watched a friend get rich off Bitcoin and you’re still kicking yourself for not doing the same. Read m…


What is the polarity of this article? nnn


Not a valid response, please try again.
What is the polarity of this article? nn


What Is Blockchain?
All of a sudden, blockchain is everywhere. The technology, which was invented i

In [21]:
t = [x for x in data if x.polarity != 'nn']

[<__main__.Struct at 0x7fca6814b2b0>, <__main__.Struct at 0x7fca6814bbe0>]